# 02 — Ejecutar E0 (ablación FS0/FS1/FS2)

Este notebook ejecuta el experimento **E0** para comparar el impacto de diferentes conjuntos de features (FS0/FS1/FS2) sobre modelos deep globales (LSTM / Transformer).

## Split
Usamos un split temporal global:
- `val_weeks = 8`
- `test_weeks = 39` (coherente con la evaluación final del proyecto)

Estrategia de evaluación:
1) Entrenar con **train** y evaluar en **val**.
2) Reentrenar con **train+val** y evaluar en **test**.

## Anti-leakage
- Features causales (lags/rollings) vienen de `src.common.make_features`.
- En modelos deep, el escalado se ajusta con el dataframe de entrenamiento del paso correspondiente.

**Requisito**: ejecuta antes el notebook 01 para generar `outputs/E0_ablation/feature_sets.json`.

In [1]:
from __future__ import annotations



import json

import time

import sys

from pathlib import Path



import pandas as pd



# Ensure PROJECT_ROOT is on sys.path so `import src.*` works reliably

NOTEBOOK_DIR = Path.cwd()

PROJECT_ROOT = NOTEBOOK_DIR

if (PROJECT_ROOT / 'src').exists() is False and (PROJECT_ROOT.parent / 'src').exists():

    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))



from src.e0_ablation_utils import (

    collect_versions,

    get_project_paths,

    get_torch_device,

    set_global_seed,

)



paths = get_project_paths(project_root=PROJECT_ROOT, output_dir='outputs/E0_ablation')

DATA_PATH = paths.data_path

OUTPUT_DIR = paths.output_dir



SEED = 42

DEBUG = True  # [COMPLETAR: pon False para el run completo]



seed_info = set_global_seed(SEED, deterministic=False)

device, device_details = get_torch_device(prefer_cuda=True)



print('PROJECT_ROOT:', PROJECT_ROOT)

print('DATA_PATH:', DATA_PATH)

print('OUTPUT_DIR:', OUTPUT_DIR)

print('device:', device, device_details)

print('seed:', seed_info)


PROJECT_ROOT: /mnt/custom-file-systems/s3/shared/TFMAXEL
DATA_PATH: /mnt/custom-file-systems/s3/shared/TFMAXEL/data/Walmart_Sales.csv
OUTPUT_DIR: /mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/E0_ablation
device: cuda {'torch_cuda_available': True, 'cuda_device_name': 'Tesla T4', 'cuda_device_count': 1, 'cuda_capability': (7, 5)}
seed: {'seed': 42, 'deterministic': False, 'numpy': 'ok', 'torch': 'ok'}


In [2]:
# Load feature set specification from notebook 01
feature_sets_path = OUTPUT_DIR / 'feature_sets.json'
if not feature_sets_path.exists():
    raise FileNotFoundError(
        f'Missing {feature_sets_path}. Run 01_data_and_feature_sets.ipynb first.'
    )

feature_sets = json.loads(feature_sets_path.read_text(encoding='utf-8'))
print('Loaded feature sets:', list(feature_sets.keys()))

# quick peek
for k in ['FS0','FS1','FS2']:
    print(k, 'n_features=', len(feature_sets[k]['feature_cols']))

Loaded feature sets: ['FS0', 'FS1', 'FS2', 'COMPLETAR']
FS0 n_features= 11
FS1 n_features= 14
FS2 n_features= 19


## Configuración del experimento
Ajusta hiperparámetros aquí. En modo `DEBUG=True`, se reducen epochs y/o se filtran tiendas para acelerar.

In [3]:
from src.common import DEFAULT_LAGS, DEFAULT_ROLLINGS, EXOG_COLUMNS, load_data, make_features, temporal_split

VAL_WEEKS = 8
TEST_WEEKS = 39

MODEL_SPECS = [
    # (name, constructor)
    ('lstm_exog', 'LSTMForecaster'),
    ('transformer_exog', 'TransformerForecaster'),
]

# Deep model training params
TRAINING_CFG = {
    'lookback': 52,
    'epochs': 3 if DEBUG else 20,
    'batch_size': 64,
    'lr': 1e-3,
    'suppress_lookback_warning': False,
}

BASE_CFG = {
    'lags': list(DEFAULT_LAGS),
    'rollings': list(DEFAULT_ROLLINGS),
}

df = load_data(DATA_PATH)
if DEBUG:
    # speed-up: keep only a few stores
    keep_stores = sorted(df['Store'].unique())[:5]
    df = df[df['Store'].isin(keep_stores)].copy()
    print('DEBUG stores:', keep_stores)

print('df shape:', df.shape)
print('date range:', df['Date'].min(), '→', df['Date'].max())

DEBUG stores: [1, 2, 3, 4, 5]
df shape: (715, 8)
date range: 2010-02-05 00:00:00 → 2012-10-26 00:00:00


In [4]:
# Split temporal global (train/val/test)
train_raw, val_raw, test_raw, split_cfg = temporal_split(df, val_weeks=VAL_WEEKS, test_weeks=TEST_WEEKS)
print(split_cfg.as_dict())

# sanity: no overlap
assert train_raw['Date'].max() < val_raw['Date'].min()
assert val_raw['Date'].max() < test_raw['Date'].min()
print('train/val/test shapes:', train_raw.shape, val_raw.shape, test_raw.shape)

{'train_start': '2010-02-05', 'train_end': '2011-12-02', 'val_start': '2011-12-09', 'val_end': '2012-01-27', 'test_start': '2012-02-03', 'test_end': '2012-10-26'}
train/val/test shapes: (480, 8) (40, 8) (195, 8)


## Runner
Ejecuta una combinación (modelo, feature set), guarda predicciones y métricas en `outputs/E0_ablation/<model>__<FS>/...`

In [5]:
from src.experiments import _compute_metrics_frames

from src.models.lstm_forecaster import LSTMForecaster
from src.models.transformer_forecaster import TransformerForecaster

MODEL_CTORS = {
    'LSTMForecaster': LSTMForecaster,
    'TransformerForecaster': TransformerForecaster,
}


def run_one(model_label: str, model_ctor_name: str, fs_name: str) -> dict:
    fs = feature_sets[fs_name]

    # Build features for this FS (we still use make_features and then filter columns)
    df_feat, all_cols = make_features(df, add_calendar=bool(fs['add_calendar']))
    used_cols = list(fs['feature_cols'])

    # Sanity: required columns exist
    missing = [c for c in used_cols if c not in df_feat.columns]
    if missing:
        raise ValueError(f'Missing engineered columns for {fs_name}: {missing[:10]}')

    # Split again but on engineered df (same dates)
    train_df = df_feat[df_feat['Date'].isin(train_raw['Date'].unique())].copy()
    val_df = df_feat[df_feat['Date'].isin(val_raw['Date'].unique())].copy()
    test_df = df_feat[df_feat['Date'].isin(test_raw['Date'].unique())].copy()

    # Config passed to model
    cfg = {
        **BASE_CFG,
        **TRAINING_CFG,
        'add_calendar': bool(fs['add_calendar']),
        'exog_cols': list(fs['exog_cols']),
        'feature_cols': used_cols,
    }

    run_dir = OUTPUT_DIR / f'{model_label}__{fs_name}'
    (run_dir / 'predictions').mkdir(parents=True, exist_ok=True)
    (run_dir / 'metrics').mkdir(parents=True, exist_ok=True)
    (run_dir / 'figures').mkdir(parents=True, exist_ok=True)

    # 1) Train -> predict val
    t0 = time.time()
    model = MODEL_CTORS[model_ctor_name]()
    model.fit(train_df, cfg)
    pred_val = model.predict(train_df, val_df, cfg)
    val_sec = time.time() - t0

    pred_val = pred_val.merge(
        val_df[['Store','Date','Weekly_Sales']].rename(columns={'Weekly_Sales':'y_true'}),
        on=['Store','Date'],
        how='left',
    )
    pred_val = pred_val.rename(columns={'y_pred':'y_pred'})
    pred_val['model'] = model_label
    pred_val['feature_set'] = fs_name

    mglob_val, mstore_val = _compute_metrics_frames(pred_val[['Store','Date','y_true','y_pred']], f'{model_label}__{fs_name}', group='VAL')
    mglob_val['feature_set'] = fs_name
    mglob_val['model'] = model_label

    pred_val.to_csv(run_dir / 'predictions' / 'val_predictions.csv', index=False)
    mglob_val.to_csv(run_dir / 'metrics' / 'val_metrics_global.csv', index=False)
    mstore_val.to_csv(run_dir / 'metrics' / 'val_metrics_by_store.csv', index=False)

    # 2) Train+Val -> predict test
    t1 = time.time()
    model2 = MODEL_CTORS[model_ctor_name]()
    trainval_df = pd.concat([train_df, val_df], ignore_index=True)
    model2.fit(trainval_df, cfg)
    pred_test = model2.predict(trainval_df, test_df, cfg)
    test_sec = time.time() - t1

    pred_test = pred_test.merge(
        test_df[['Store','Date','Weekly_Sales']].rename(columns={'Weekly_Sales':'y_true'}),
        on=['Store','Date'],
        how='left',
    )
    pred_test['model'] = model_label
    pred_test['feature_set'] = fs_name

    mglob_test, mstore_test = _compute_metrics_frames(pred_test[['Store','Date','y_true','y_pred']], f'{model_label}__{fs_name}', group='TEST')
    mglob_test['feature_set'] = fs_name
    mglob_test['model'] = model_label

    pred_test.to_csv(run_dir / 'predictions' / 'test_predictions.csv', index=False)
    mglob_test.to_csv(run_dir / 'metrics' / 'test_metrics_global.csv', index=False)
    mstore_test.to_csv(run_dir / 'metrics' / 'test_metrics_by_store.csv', index=False)

    # Save run metadata
    meta = {
        'seed': SEED,
        'debug': DEBUG,
        'model': model_label,
        'feature_set': fs_name,
        'split': split_cfg.as_dict(),
        'config': cfg,
        'device': device,
        'device_details': device_details,
        'versions': collect_versions(),
        'timing_sec': {
            'val_fit_predict': float(val_sec),
            'test_fit_predict': float(test_sec),
        },
    }
    (run_dir / 'run_metadata.json').write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding='utf-8')

    return {
        'model': model_label,
        'feature_set': fs_name,
        'val_fit_predict_sec': float(val_sec),
        'test_fit_predict_sec': float(test_sec),
        **{k: float(mglob_test.iloc[0][k]) for k in ['MAE','RMSE','sMAPE']},
    }

In [6]:
# Run grid
results = []

for model_label, ctor_name in MODEL_SPECS:
    for fs_name in ['FS0','FS1','FS2']:
        print('===', model_label, fs_name, '===')
        row = run_one(model_label=model_label, model_ctor_name=ctor_name, fs_name=fs_name)
        results.append(row)

summary = pd.DataFrame(results).sort_values(['model','feature_set']).reset_index(drop=True)
display(summary)

summary_path = OUTPUT_DIR / 'summary_metrics.csv'
summary.to_csv(summary_path, index=False)
print('Saved:', summary_path)

=== lstm_exog FS0 ===
[LSTM] Reducing lookback from 52 to 43 due to limited per-store history (min len=44 after dropna).
[LSTM] Reducing lookback from 52 to 51 due to limited per-store history (min len=52 after dropna).
=== lstm_exog FS1 ===
[LSTM] Reducing lookback from 52 to 43 due to limited per-store history (min len=44 after dropna).
[LSTM] Reducing lookback from 52 to 51 due to limited per-store history (min len=52 after dropna).
=== lstm_exog FS2 ===
[LSTM] Reducing lookback from 52 to 43 due to limited per-store history (min len=44 after dropna).
[LSTM] Reducing lookback from 52 to 51 due to limited per-store history (min len=52 after dropna).
=== transformer_exog FS0 ===
[Transformer] Reducing lookback from 52 to 43 due to limited per-store history (min len=44 after dropna).
[Transformer] Reducing lookback from 52 to 51 due to limited per-store history (min len=52 after dropna).
=== transformer_exog FS1 ===
[Transformer] Reducing lookback from 52 to 43 due to limited per-store

,model,feature_set,val_fit_predict_sec,test_fit_predict_sec,MAE,RMSE,sMAPE
0,lstm_exog,FS0,2.341277,0.318473,710089.292938,7.581639e+05,64.518722
1,lstm_exog,FS1,0.139632,0.324879,729429.159086,7.577307e+05,66.809316
2,lstm_exog,FS2,0.148299,0.336044,722022.504484,7.552919e+05,66.000368
3,transformer_exog,FS0,0.307933,0.482765,920418.052308,1.088733e+06,118.051787
4,transformer_exog,FS1,0.184602,0.488274,893289.744446,1.069371e+06,111.388046
5,transformer_exog,FS2,0.193915,0.503371,611855.154668,6.413315e+05,58.701062


Saved: /mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/E0_ablation/summary_metrics.csv


Siguiente: ejecutar **03_results_summary_and_plots.ipynb** para consolidación, deltas y visualizaciones.